In [1]:
import pandas as pd
import numpy as np

test_group_data = pd.read_csv('./data/test_format1.csv')
train_group_data = pd.read_csv('./data/train_format1.csv')
user_info_data = pd.read_csv('./data/user_info_format1.csv')
user_log_data = pd.read_csv('./data/user_log_format1.csv').rename(columns={'seller_id': 'merchant_id'})

In [3]:
user_info_data.head()

,user_id,age_range,gender
0,376517,6.0,1.0
1,234512,5.0,0.0
2,344532,5.0,0.0
3,186135,5.0,0.0
4,30230,5.0,0.0


In [4]:
user_log_data.head()

,user_id,item_id,cat_id,merchant_id,brand_id,time_stamp,action_type
0,328862,323294,833,2882,2661.0,829,0
1,328862,844400,1271,2882,2661.0,829,0
2,328862,575153,1271,2882,2661.0,829,0
3,328862,996875,1271,2882,2661.0,829,0
4,328862,1086186,1271,1253,1049.0,829,0


In [6]:
train_group_data.head()

,user_id,merchant_id,label
0,34176,3906,0
1,34176,121,0
2,34176,4356,1
3,34176,2217,0
4,230784,4818,0


## 特征工程
第一，将time_stamp字段补全为完整的时间格式，并基于这个时间戳转换成季度，月，周，日的信息。

In [5]:
import datetime
user_log_data['time_stamp'] = [datetime.date(2015, int(str(i)[:-2]), int(str(i)[-2:]))
                                    for i in user_log_data['time_stamp']]
user_log_data['weekday'] = [i.weekday() for i in user_log_data['time_stamp']]
user_log_data['month'] = [i.month for i in user_log_data['time_stamp']]
user_log_data['season'] = ['summer' if i in [5, 6, 7] else 'autumn' for i in user_log_data['month']]
user_log_data['day'] = [i.day for i in user_log_data['time_stamp']]
# 以2015年一月一号为起点，计算出当前时间戳距离这个起点的日数
user_log_data['dayCount'] = [(i - datetime.date(2015, 1, 1)).days for i in user_log_data['time_stamp']]

第二，构造出用户的行为特征。

In [21]:
from tqdm import tqdm
user_feature_record_map = {}
total_user_id_set = set(train_group_data['user_id']) | set(test_group_data['user_id'])
for user_id in tqdm(list(total_user_id_set)):
    # 锁定502用户进行测试
    user_id = 502
    print('正在处理用户：', user_id)
    user_id_data = user_log_data[user_log_data['user_id'] == user_id]
    # 该用户的年龄和性别
    _, user_age, user_gender = user_info_data[user_info_data['user_id']==user_id].values[0]
    # 该用户的总点击行为，加购行为，购买行为和收藏行为数量
    action_type_count = user_id_data['action_type'].value_counts()
    user_action_click = action_type_count.get(0, 0)
    user_action_toCart = action_type_count.get(1, 0)
    user_action_purchase = action_type_count.get(2, 0)
    user_action_collect = action_type_count.get(3, 0)
    # 该用户的点击转化率，加购转化率和收藏转化率
    user_action_click_to_purchase_rate = user_action_click / user_action_purchase
    user_action_toCart_to_purchase_rate = user_action_toCart / user_action_purchase
    user_action_collect_to_purchase_rate = user_action_collect / user_action_purchase
    # 计算出该用户在一周7天的活跃次数以及比例情况，这其中可能反应用户的作息规律和职业特性
    weekday_active_count = user_id_data.groupby('weekday')['user_id'].count()
    user_active_count_in_week = [weekday_activate_count.get(i, 0) for i in range(7)]
    user_active_ratio_in_week = [i / max(1, sum(user_active_count_in_week)) for i in user_active_count_in_week]
    # 计算出该用户在5,6,7,8,9,10,11月分别的活跃次数和活跃比率
    month_active_count = user_id_data.groupby('month')['user_id'].count()
    user_active_count_in_mouth = [month_active_count.get(i, 0) for i in [5, 6, 7, 8, 9, 10, 11]]
    user_active_ratio_in_mouth = [i / max(1, sum(user_active_count_in_mouth)) for i in user_active_count_in_mouth]
    # 计算出消费者在双十一活动前n天的活跃次数和活跃比率（这里的n需要根据业务经验来确定）
    # 你来完成这个指标的计算
    # Code Here
    # 消费者行为涉及的具体商品个数（这里的消费者行为包括全部的action_type）
    user_touch_item_num = len(set(user_id_data['item_id'].values))
    # 消费者行为涉及的类别个数
    # Code Here
    # 消费者行为涉及的商家个数
    # Code Here
    # 消费者行为涉及的品牌个数
    # Code Here
    #消费者复购的商品个数，类别个数，商家个数，品牌个数
    Code
    break

  0%|                                                                                       | 0/424170 [00:00<?, ?it/s]

正在处理用户： 502


In [27]:
user_id_data.groupby('weekday')['user_id'].count()

weekday
0    19
1    12
2    14
3     3
4     2
5     1
6     2
Name: user_id, dtype: int64